In [1]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance
from geopy.distance import great_circle
import random
import pickle

In [2]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()

In [3]:
print(wells_oilgas.WellType.value_counts())
wells_oilgas.head()

OG     158064
DG       5003
GAS      1417
LG          6
Name: WellType, dtype: int64


,OID,API,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,FieldName,...,Township,Range,BaseMeridi,Latitude,Longitude,GISSource,isConfiden,isDirectio,SpudDate,WellSymbol
1,-1,402100016,Afton Community 1,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.451515,-121.950668,hud,N,N,12/02/1943,PluggedDG
4,-1,402100017,Afton Community 2,1,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460217,-121.953537,hud,N,N,04/14/1945,PluggedDG
5,-1,402120283,Afton Community 2,2,Plugged,DG,Dry Gas,V1150,"Venoco, LLC",Afton Gas,...,19N,01W,MD,39.460033,-121.951424,hud,N,N,10/10/1984,PluggedDG
8,-1,402120815,Anacapa,3-4,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.443230,-121.951042,GPS,N,N,11/06/2002,IdleDG
9,-1,402120494,Andreotti,15-26,Plugged,DG,Dry Gas,W1700,Western Continental Operating Company,Afton Gas,...,19N,01W,MD,39.470551,-121.947769,GPS,N,N,10/05/1987,PluggedDG


In [4]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

facilities = facilities[facilities.State == 'CA']

In [5]:
facilities.shape

(382, 67)

In [6]:
facilitiesB = facilities.rename(inplace=False,
                                columns={'Facility Name':'facility_name',
                                         'City':'city',
                                         'Primary NAICS Code':'naics_code',
                                         'Industry Type (subparts)':'industry_subparts',
                                         'Industry Type (sectors)':'industry_sectors'})

emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
#                                               'Facility Name2',
                                               'city',
                                               'Zip Code',
                                               'Address',
                                               'County',
                                               'Latitude',
                                               'Longitude',
#                                               'Primary NAICS Code2',
                                               'industry_subparts',
                                               'industry_sectors']],
                        how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude',
                                'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                    'Facility Id').agg({'Latitude':'min',
                                                        'Longitude':'min',
                                                        'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                  'Latitude':'lat',
                                                  'Longitude':'lon',
                                                  'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg.head()


,facility_id,lat,lon,annual_emissions
0,1000005,36.923750,-120.104120,60760.1
2,1000051,37.796500,-121.604900,52901.5
3,1000064,35.447778,-119.260000,64051.2
4,1000066,37.074790,-120.877445,69721.9
5,1000070,34.595300,-117.364700,1357906.4


In [7]:
facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
facility_ids = emsCA2019locagg['facility_id'].tolist()

In [8]:
facility_latlon.shape

(288,)

In [8]:
# This cell takes ages to run - it's creating a full cartesian matrix of distances between facilities and wells
x = time()
distance_matrix = []
for i in facility_latlon:
    distances = []
    for j in wells_latlon:
        dist = distance(i,j).miles
        distances.append(dist)
    distance_matrix.append(distances)

print(time() - x, 'seconds')

7766.696553945541 seconds


In [29]:
# This cell takes ages to run - it's creating a full cartesian matrix of distances between facilities and wells
x = time()
distance_matrix2 = np.empty(dtype=float,
                           shape=[len(facility_latlon), len(wells_latlon)])
for i in range(len(facility_latlon)):
    for j in range(len(wells_latlon)):
        distance_matrix2[i,j] = distance(facility_latlon[i],
                                         wells_latlon[j]).miles
print(time() - x, 'seconds')

12437.3069794178 seconds


In [10]:
# This cell takes ages to run - it's creating a full cartesian matrix of distances between facilities and wells
x = time()
distance_matrix3 = np.empty(dtype=float,
                           shape=[len(facility_latlon), len(wells_latlon)])
for i in range(len(facility_latlon)):
    for j in range(len(wells_latlon)):
        distance_matrix3[i,j] = great_circle(facility_latlon[i],
                                             wells_latlon[j]).miles
print(time() - x, 'seconds')

17609.400146961212 seconds


In [9]:
# Generate random numbers for demand volumes, to be changed
well_volumes = [random.uniform(100,1000) for i in wells_ids]
well_volumes_uniform = [500]*len(wells_ids)


In [10]:
len(wells_ids)

164490

In [11]:
prob = LpProblem('Unbalaced Transportation Problem', LpMinimize)
n_receivers = len(wells_ids)
n_suppliers = len(facility_ids)
supply = emsCA2019locagg['annual_emissions'].tolist()
demand = well_volumes_uniform
costs = distance_matrix

/Users/emmatebbe/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [12]:
# Takes a long time to run because it's solving the problem
routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

x = LpVariable.dicts('X', routes, lowBound=0)
prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

for i in range(n_suppliers):
    prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply[i]
    
for j in range(n_receivers):
    prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand[j]
    
# Solving problem
prob.solve()


1

In [ ]:
# Also takes forever to run

print('Status', LpStatus[prob.status])

print('Z = {}'.format(value(prob.objective)))

In [13]:

for v in prob.variables():
    if (v.varValue > 0):
        print('{} = {}'.format(v.name, v.varValue))

X_(0,_140773) = 500.0
X_(0,_140775) = 500.0
X_(0,_140776) = 500.0
X_(0,_140777) = 500.0
X_(0,_140778) = 500.0
X_(0,_140779) = 500.0
X_(0,_140780) = 500.0
X_(0,_140781) = 500.0
X_(0,_140782) = 500.0
X_(0,_140783) = 500.0
X_(0,_140784) = 500.0
X_(0,_140785) = 500.0
X_(0,_140786) = 500.0
X_(0,_140787) = 500.0
X_(0,_140788) = 500.0
X_(0,_140789) = 500.0
X_(0,_140790) = 500.0
X_(0,_140791) = 500.0
X_(0,_140792) = 500.0
X_(0,_140793) = 500.0
X_(0,_140794) = 500.0
X_(0,_140795) = 500.0
X_(0,_140796) = 500.0
X_(0,_140798) = 500.0
X_(0,_140799) = 500.0
X_(0,_140800) = 500.0
X_(0,_140801) = 500.0
X_(0,_140802) = 500.0
X_(0,_140803) = 500.0
X_(0,_140805) = 500.0
X_(0,_140806) = 500.0
X_(0,_140807) = 500.0
X_(0,_140808) = 500.0
X_(0,_140809) = 500.0
X_(0,_140810) = 500.0
X_(0,_140811) = 500.0
X_(0,_140812) = 500.0
X_(0,_140813) = 500.0
X_(0,_140814) = 500.0
X_(0,_140815) = 500.0
X_(0,_140816) = 500.0
X_(0,_140817) = 500.0
X_(0,_140818) = 500.0
X_(0,_140819) = 500.0
X_(0,_140820) = 500.0
X_(0,_1408

X_(101,_132747) = 500.0
X_(101,_132748) = 500.0
X_(101,_132757) = 500.0
X_(101,_132761) = 500.0
X_(101,_132766) = 500.0
X_(101,_132767) = 500.0
X_(101,_132768) = 500.0
X_(101,_132769) = 500.0
X_(101,_132770) = 500.0
X_(101,_132771) = 500.0
X_(101,_132772) = 500.0
X_(101,_132773) = 500.0
X_(101,_132775) = 500.0
X_(101,_132777) = 500.0
X_(101,_132778) = 500.0
X_(101,_132779) = 500.0
X_(101,_132781) = 500.0
X_(101,_159719) = 500.0
X_(101,_159720) = 115.84
X_(101,_159730) = 500.0
X_(101,_159731) = 500.0
X_(101,_159738) = 500.0
X_(101,_159739) = 500.0
X_(101,_159740) = 500.0
X_(101,_159741) = 500.0
X_(101,_159743) = 500.0
X_(101,_159744) = 500.0
X_(101,_159745) = 500.0
X_(101,_159746) = 500.0
X_(101,_159747) = 500.0
X_(101,_159748) = 500.0
X_(101,_159749) = 500.0
X_(101,_159750) = 500.0
X_(101,_159753) = 500.0
X_(101,_159754) = 500.0
X_(101,_160086) = 500.0
X_(101,_160093) = 500.0
X_(101,_160109) = 500.0
X_(101,_160110) = 500.0
X_(101,_160111) = 500.0
X_(101,_160112) = 500.0
X_(101,_160113)

X_(103,_72564) = 500.0
X_(103,_72594) = 500.0
X_(103,_72595) = 500.0
X_(103,_72728) = 500.0
X_(103,_72733) = 500.0
X_(103,_72846) = 500.0
X_(103,_72847) = 500.0
X_(103,_74184) = 106.4
X_(103,_74195) = 500.0
X_(103,_74200) = 500.0
X_(103,_74205) = 500.0
X_(103,_74216) = 500.0
X_(103,_74239) = 500.0
X_(103,_74253) = 500.0
X_(103,_74300) = 500.0
X_(103,_74301) = 500.0
X_(104,_63355) = 500.0
X_(104,_63371) = 500.0
X_(104,_63372) = 500.0
X_(104,_63373) = 500.0
X_(104,_63374) = 500.0
X_(104,_63386) = 500.0
X_(104,_63400) = 500.0
X_(104,_63401) = 500.0
X_(104,_63402) = 500.0
X_(104,_63403) = 500.0
X_(104,_63418) = 500.0
X_(104,_63419) = 500.0
X_(104,_63420) = 500.0
X_(104,_63421) = 500.0
X_(104,_63422) = 500.0
X_(104,_63423) = 500.0
X_(104,_63424) = 500.0
X_(104,_63426) = 500.0
X_(104,_63427) = 500.0
X_(104,_63428) = 500.0
X_(104,_63429) = 500.0
X_(104,_63430) = 500.0
X_(104,_63431) = 500.0
X_(104,_63451) = 500.0
X_(104,_63453) = 500.0
X_(104,_63454) = 500.0
X_(104,_63455) = 500.0
X_(104,_634

X_(110,_136361) = 500.0
X_(110,_137459) = 500.0
X_(110,_63319) = 500.0
X_(110,_71925) = 500.0
X_(110,_72023) = 354.2
X_(110,_72030) = 500.0
X_(110,_72031) = 500.0
X_(110,_72040) = 500.0
X_(110,_72041) = 500.0
X_(110,_72096) = 500.0
X_(110,_72128) = 500.0
X_(110,_72141) = 500.0
X_(110,_72338) = 500.0
X_(110,_72347) = 500.0
X_(111,_136336) = 219.1
X_(111,_136354) = 500.0
X_(111,_137069) = 500.0
X_(111,_137460) = 500.0
X_(111,_63271) = 302.1
X_(111,_63313) = 500.0
X_(111,_63314) = 500.0
X_(111,_63317) = 500.0
X_(111,_73012) = 500.0
X_(111,_73213) = 500.0
X_(111,_74092) = 500.0
X_(112,_72947) = 410.8
X_(112,_72956) = 500.0
X_(112,_72959) = 500.0
X_(112,_72972) = 500.0
X_(112,_72973) = 500.0
X_(112,_72974) = 500.0
X_(112,_72975) = 500.0
X_(112,_72990) = 500.0
X_(112,_72993) = 500.0
X_(112,_72997) = 500.0
X_(112,_72998) = 500.0
X_(112,_72999) = 500.0
X_(112,_73004) = 500.0
X_(112,_73005) = 500.0
X_(112,_73006) = 500.0
X_(112,_73007) = 500.0
X_(112,_73008) = 500.0
X_(112,_73010) = 500.0
X_(11

X_(12,_104710) = 500.0
X_(12,_104711) = 500.0
X_(12,_105014) = 500.0
X_(12,_105015) = 500.0
X_(12,_105048) = 500.0
X_(12,_105767) = 500.0
X_(12,_105768) = 500.0
X_(12,_105769) = 500.0
X_(12,_1076) = 500.0
X_(12,_109035) = 500.0
X_(12,_109226) = 500.0
X_(12,_109449) = 500.0
X_(12,_109450) = 500.0
X_(12,_109451) = 500.0
X_(12,_109785) = 500.0
X_(12,_109797) = 500.0
X_(12,_110241) = 500.0
X_(12,_110245) = 500.0
X_(12,_110248) = 500.0
X_(12,_110250) = 500.0
X_(12,_1107) = 500.0
X_(12,_111133) = 500.0
X_(12,_111167) = 500.0
X_(12,_111274) = 500.0
X_(12,_111286) = 500.0
X_(12,_111293) = 500.0
X_(12,_111311) = 500.0
X_(12,_111316) = 500.0
X_(12,_111323) = 500.0
X_(12,_111330) = 500.0
X_(12,_111337) = 500.0
X_(12,_111339) = 500.0
X_(12,_111341) = 500.0
X_(12,_111342) = 500.0
X_(12,_111378) = 500.0
X_(12,_111381) = 500.0
X_(12,_111387) = 500.0
X_(12,_111390) = 500.0
X_(12,_111393) = 500.0
X_(12,_111397) = 500.0
X_(12,_111404) = 500.0
X_(12,_111408) = 500.0
X_(12,_111409) = 500.0
X_(12,_111433) 

...

X_(84,_134635) = 500.0
X_(84,_134636) = 500.0
X_(84,_134637) = 500.0
X_(84,_134638) = 500.0
X_(84,_134643) = 500.0
X_(84,_134699) = 500.0
X_(84,_134733) = 500.0
X_(84,_134734) = 500.0
X_(84,_134744) = 500.0
X_(84,_134768) = 500.0
X_(84,_134774) = 500.0
X_(84,_134777) = 500.0
X_(84,_134778) = 500.0
X_(84,_134779) = 500.0
X_(84,_134780) = 500.0
X_(84,_134781) = 500.0
X_(84,_134782) = 500.0
X_(84,_134790) = 500.0
X_(84,_134803) = 500.0
X_(84,_134806) = 500.0
X_(84,_134807) = 500.0
X_(84,_134814) = 500.0
X_(84,_134835) = 500.0
X_(84,_134858) = 500.0
X_(84,_134859) = 500.0
X_(84,_134860) = 500.0
X_(84,_1365) = 500.0
X_(84,_1415) = 500.0
X_(84,_1531) = 500.0
X_(84,_1579) = 500.0
X_(84,_158131) = 500.0
X_(84,_158135) = 500.0
X_(84,_1866) = 500.0
X_(84,_2131) = 500.0
X_(84,_2338) = 500.0
X_(84,_2339) = 500.0
X_(84,_2388) = 500.0
X_(84,_38876) = 500.0
X_(84,_56933) = 500.0
X_(84,_56934) = 500.0
X_(84,_56935) = 500.0
X_(84,_56936) = 500.0
X_(84,_66142) = 500.0
X_(84,_717) = 500.0
X_(84,_729) = 5

X_(87,_2398) = 500.0
X_(87,_46447) = 500.0
X_(87,_46465) = 500.0
X_(87,_46466) = 500.0
X_(87,_46467) = 500.0
X_(87,_46468) = 500.0
X_(87,_46469) = 500.0
X_(87,_46470) = 500.0
X_(87,_46471) = 500.0
X_(87,_46472) = 500.0
X_(87,_46473) = 500.0
X_(87,_46474) = 500.0
X_(87,_46475) = 500.0
X_(87,_46476) = 500.0
X_(87,_46477) = 500.0
X_(87,_46478) = 500.0
X_(87,_46479) = 500.0
X_(87,_46480) = 500.0
X_(87,_46481) = 500.0
X_(87,_46482) = 500.0
X_(87,_46483) = 500.0
X_(87,_46484) = 500.0
X_(87,_46485) = 500.0
X_(87,_46486) = 500.0
X_(87,_46487) = 500.0
X_(87,_46488) = 500.0
X_(87,_46489) = 500.0
X_(87,_46490) = 500.0
X_(87,_46491) = 500.0
X_(87,_46498) = 500.0
X_(87,_46499) = 500.0
X_(87,_46559) = 500.0
X_(87,_46565) = 500.0
X_(87,_46567) = 500.0
X_(87,_46568) = 500.0
X_(87,_46573) = 500.0
X_(87,_46577) = 500.0
X_(87,_46626) = 500.0
X_(87,_46627) = 500.0
X_(87,_46629) = 500.0
X_(87,_46630) = 500.0
X_(87,_46632) = 500.0
X_(87,_46637) = 500.0
X_(87,_46639) = 500.0
X_(87,_46645) = 500.0
X_(87,_4664

X_(90,_1602) = 26.3
X_(90,_1710) = 500.0
X_(90,_83369) = 500.0
X_(90,_83370) = 500.0
X_(90,_83371) = 500.0
X_(90,_83372) = 500.0
X_(90,_83373) = 500.0
X_(90,_83374) = 500.0
X_(90,_83375) = 500.0
X_(90,_83376) = 500.0
X_(91,_157) = 500.0
X_(91,_165) = 500.0
X_(91,_180) = 500.0
X_(91,_182) = 500.0
X_(91,_183) = 500.0
X_(91,_186) = 500.0
X_(91,_187) = 500.0
X_(91,_188) = 500.0
X_(91,_190) = 500.0
X_(91,_191) = 500.0
X_(91,_193) = 500.0
X_(91,_195) = 500.0
X_(91,_196) = 500.0
X_(91,_197) = 500.0
X_(91,_200) = 500.0
X_(91,_201) = 500.0
X_(91,_203) = 500.0
X_(91,_207) = 500.0
X_(91,_209) = 500.0
X_(91,_210) = 500.0
X_(91,_211) = 500.0
X_(91,_212) = 500.0
X_(91,_213) = 500.0
X_(91,_71414) = 500.0
X_(91,_71511) = 500.0
X_(91,_76233) = 500.0
X_(91,_76251) = 500.0
X_(91,_76253) = 500.0
X_(91,_76258) = 500.0
X_(91,_76259) = 500.0
X_(91,_76576) = 500.0
X_(91,_76668) = 500.0
X_(91,_76669) = 500.0
X_(91,_76670) = 500.0
X_(91,_76671) = 500.0
X_(91,_76675) = 500.0
X_(91,_76676) = 500.0
X_(91,_76679) =

X_(95,_54552) = 124.34
X_(95,_54592) = 209.96
X_(95,_56066) = 500.0
X_(95,_56067) = 500.0
X_(95,_56093) = 500.0
X_(95,_56094) = 500.0
X_(95,_56095) = 500.0
X_(95,_56096) = 500.0
X_(95,_56098) = 500.0
X_(95,_56349) = 500.0
X_(95,_56354) = 500.0
X_(95,_56361) = 500.0
X_(95,_56382) = 500.0
X_(95,_56383) = 500.0
X_(95,_56395) = 500.0
X_(95,_56768) = 500.0
X_(95,_56769) = 500.0
X_(95,_56771) = 500.0
X_(95,_56773) = 500.0
X_(95,_56774) = 500.0
X_(95,_56775) = 500.0
X_(95,_56776) = 500.0
X_(95,_56778) = 500.0
X_(95,_56780) = 500.0
X_(95,_56781) = 500.0
X_(95,_56786) = 500.0
X_(95,_56801) = 500.0
X_(96,_3691) = 500.0
X_(96,_96239) = 500.0
X_(96,_96406) = 383.6
X_(96,_96407) = 500.0
X_(96,_96422) = 500.0
X_(96,_96427) = 500.0
X_(96,_96428) = 500.0
X_(96,_96442) = 500.0
X_(96,_96444) = 500.0
X_(96,_96445) = 500.0
X_(96,_96449) = 500.0
X_(96,_96457) = 500.0
X_(96,_96458) = 500.0
X_(96,_96462) = 500.0
X_(96,_96463) = 500.0
X_(96,_97078) = 500.0
X_(96,_97406) = 500.0
X_(96,_97647) = 500.0
X_(96,_97

X_(99,_1025) = 500.0
X_(99,_1109) = 500.0
X_(99,_1702) = 500.0
X_(99,_1703) = 500.0
X_(99,_1704) = 500.0
X_(99,_2116) = 500.0
X_(99,_35793) = 500.0
X_(99,_35794) = 500.0
X_(99,_35795) = 500.0
X_(99,_35796) = 500.0
X_(99,_35797) = 500.0
X_(99,_35798) = 500.0
X_(99,_35799) = 500.0
X_(99,_35800) = 500.0
X_(99,_35801) = 500.0
X_(99,_35802) = 500.0
X_(99,_35803) = 500.0
X_(99,_35804) = 500.0
X_(99,_35805) = 500.0
X_(99,_64350) = 500.0
X_(99,_64351) = 500.0
X_(99,_64360) = 500.0
X_(99,_64361) = 500.0
X_(99,_64367) = 500.0
X_(99,_64377) = 500.0
X_(99,_64378) = 500.0
X_(99,_64379) = 500.0
X_(99,_64383) = 500.0
X_(99,_64385) = 500.0
X_(99,_64386) = 500.0
X_(99,_64393) = 500.0
X_(99,_64394) = 500.0
X_(99,_64396) = 500.0
X_(99,_64397) = 500.0
X_(99,_64398) = 500.0
X_(99,_64400) = 500.0
X_(99,_64401) = 500.0
X_(99,_64402) = 500.0
X_(99,_64403) = 500.0
X_(99,_64405) = 500.0
X_(99,_64406) = 500.0
X_(99,_64408) = 500.0
X_(99,_64411) = 500.0
X_(99,_64412) = 500.0
X_(99,_64413) = 500.0
X_(99,_64414) = 

In [14]:
volumes_moved = [i.varValue for i in prob.variables()]


In [15]:
total_volume = 0
for i in range(0, len(volumes_moved)):
    total_volume = total_volume + volumes_moved[i]
print(total_volume)

59634958.98999999


In [16]:
volume_matrix = []
k=0
while k < n_suppliers:
    vols = []
    for i in range(n_receivers):
        j = i + k*n_receivers
        vols.append(volumes_moved[j])
    volume_matrix.append(vols)
    k+=1

In [17]:
print(len(volume_matrix))
print(len(volume_matrix[0]))

288
164490


In [18]:
print(len(distance_matrix))
print(len(distance_matrix[0]))

288
164490


In [19]:
results = []
for i in range(n_suppliers):
    f_loc = facility_latlon[i]
    for j in range(n_receivers):
        vol = volume_matrix[i][j]
        if vol > 0:
            w_loc = wells_latlon[j]
            dist = distance_matrix[i][j]
            r = [f_loc, w_loc, dist, vol]
            results.append(r)

In [20]:
file_name = 'data/results/results_toydata_uniform.pkl'
open_file = open(file_name, "wb")
pickle.dump(results, open_file)
open_file.close()

In [ ]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_toydata.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

